In [255]:
import k3d
import os
import vtk

from k3d.helpers import download

import speech_recognition as sr
import requests
import json

import ipywidgets as widgets
from IPython.display import display

r = sr.Recognizer()
plot = k3d.plot()
plot.display()

rendered_objs = []

import gensim
from pprint import pprint

Output()

In [ ]:
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('model/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [113]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [186]:
sentence = 'chair' 

my_phrases = [{x.pos_: x.lemma_} for x in [y 
                                      for y
                                      in nlp(str(sentence)) 
                                      if not y.is_stop and y.pos_ == 'NOUN']]

In [274]:
def getMeText():
    textPart = ''
    with sr.Microphone() as source:
        print("Say something!")
        audio = r.listen(source,phrase_time_limit=3)
    try:
        textPart = r.recognize_google(audio)
        print("Detected Text: ", textPart) 
    except sr.UnknownValueError:
        textPart = ''
        print("Click and Speak again")
        return 
    except sr.RequestError as e:
        textPart = ''
        print("Click and Speak again")
        return
    infer_input_instructions(textPart)

    
def infer_input_instructions(text):
    if "clear" in text or "clean" in text:
        remove_objs()
        return
    else:
        my_phrases = [{x.pos_: x.lemma_.lower()} for x in [y 
                                          for y
                                          in nlp(str(text)) 
                                          if not y.is_stop and y.pos_ == 'NOUN']]
        # print("Phrases: ",my_phrases)
        if len(my_phrases) > 0: 
            my_object = my_phrases[0]['NOUN'].lower()
            render_k3d_obj(my_object)
        else:
            print("\nNO OBJECT OF THIS NAME FOUND IN DATABASE WHILE INFERING, PLEASE TRY ANOTHER QUERY!")
            return
        

def render_k3d_obj(text):
    
    if 'chair' in text:
        path = 'objects/my_chair.obj'
        model_matrix = (
             0.75,  0.0, 0.0, 0,
             0.0,  0.0, 0.75, 50,
             0.0,  0.75, 0.0, 0.0,
             100.0,  100.0, 100.0, 250.0
        )
    elif 'man' in text or 'person' in text:
        path = 'objects/smokingMan.obj'
        model_matrix = (
             0.65,  0.0, 0.0, -100,
             0.0, 0.0, 0.65, 0,
             0.0,  0.65, 0.0, 0,
             100.0,  100.0, 100.0, 250.0
        )
    elif 'table' in text:
        path = 'objects/my_table.obj'
        model_matrix = (
              1.0,  0.0, 0.0, 0.0,
                 0.0,  0.0, 1.0, 0.0,
                 0.0,  1.0, 0.0, 0.0,
                 0.0,  0.0, 0.0, 1.0
        )
    else:
        print("\nNO OBJECT FOUND IN DATABASE WHILE RENDERING, PLEASE TRY ANOTHER QUERY!")
        return
    
    print("Rendering: %s"%text)
    reader = vtk.vtkOBJReader()
    reader.SetFileName(path)
    reader.Update()
    
    my_3d_obj = k3d.vtk_poly_data(reader.GetOutput(), color=0xff0000, model_matrix=model_matrix)
    my_3d_obj.wireframe = False
    rendered_objs.append(my_3d_obj)
    global plot
    plot += my_3d_obj
    
    print("Similar items for this object:")
    print("***********************************")
    pprint(model.most_similar([text]))
    print("***********************************")
    
def remove_objs():
    global rendered_objs
    for obj in rendered_objs:
        global plot
        plot -= obj
    rendered_objs = []

In [279]:
remove_objs()

In [280]:
render_k3d_obj('table')

Rendering: table
Similar items for this object:
***********************************
[('tables', 0.695063591003418),
 ('ConocoPhillips_BPAmerica', 0.4951048493385315),
 ('Capitalized_Included', 0.46665361523628235),
 ('tray', 0.46534568071365356),
 ('dining_room', 0.45831137895584106),
 ('banquette', 0.4455730617046356),
 ('rapping_cappella', 0.4435313045978546),
 ('sideboard', 0.44340980052948),
 ('linen_tablecloth', 0.4422051012516022),
 ('Tables', 0.44195544719696045)]
***********************************


In [281]:
render_k3d_obj('chair')

Rendering: chair
Similar items for this object:
***********************************
[('chairs', 0.7871319055557251),
 ('Chair', 0.7488438487052917),
 ('chairperson', 0.6301863193511963),
 ('chairwoman', 0.6271359920501709),
 ('chairman', 0.616144061088562),
 ('Vice_Chair', 0.5646666288375854),
 ('Co_Chair', 0.5469809770584106),
 ('chairing', 0.5431761741638184),
 ('Chairs', 0.5395591259002686),
 ('cochair', 0.5284922122955322)]
***********************************


In [282]:
render_k3d_obj('person')

Rendering: person
Similar items for this object:
***********************************
[('someone', 0.6657355427742004),
 ('persons', 0.5559710264205933),
 ('woman', 0.5470173358917236),
 ('somebody', 0.5459041595458984),
 ('peson', 0.5421414375305176),
 ('man', 0.5342026948928833),
 ('people', 0.5083408951759338),
 ('anyone', 0.5061744451522827),
 ('guy', 0.48752474784851074),
 ('Someone', 0.473005473613739)]
***********************************


In [275]:
speak_button = widgets.Button(description="Click to speak!")
clear_button = widgets.Button(description="Clear View")

output = widgets.Output()

display(speak_button, clear_button, output)

def on_speak_button_clicked(b):
    with output:
        getMeText()
        
def on_clear_button_clicked(b):
    with output:
        remove_objs()

speak_button.on_click(on_button_clicked)
clear_button.on_click(on_clear_button_clicked)


Button(description='Click to speak!', style=ButtonStyle())

Button(description='Clear View', style=ButtonStyle())

Output()